In [27]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import statsmodels.api as sm
from sklearn import metrics as skmetrics

### Verify model with personality

In [51]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')

In [52]:
df.shape

(1039, 186)

In [53]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [54]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

### Evaluation code

In [55]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(2)
    target_names = ['no', 'yes']
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [56]:
def evaluate_prediction(predictions, target, title="Confusion matrix"):
    print('accuracy %s' % accuracy_score(target, predictions))
    cm = confusion_matrix(target, predictions)
    print('confusion matrix\n %s' % cm)
    print('(row=expected, col=predicted)')
    
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plot_confusion_matrix(cm_normalized, title + ' Normalized')

In [57]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

## Doc2Vec

In [58]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [59]:
def tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [60]:
train_tagged = train_w2v_data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['formatted_text']), tags=[r.extraversion_m]), axis=1)

In [61]:
test_tagged = test_w2v_data.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['formatted_text']), tags=[r.extraversion_m]), axis=1)

In [62]:
%%time
trainsent = train_tagged.values
testsent = test_tagged.values

# simple gensim doc2vec api
doc2vec_model = Doc2Vec(trainsent, workers=1, size=5, iter=20, dm=1)

train_targets, train_regressors = zip(
    *[(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in trainsent])

2017-05-11 14:07:56,983 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-05-11 14:07:56,984 : INFO : collecting all words and their counts
2017-05-11 14:07:56,985 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-11 14:07:57,306 : INFO : collected 43446 word types and 2 unique tags from a corpus of 727 examples and 1472087 words
2017-05-11 14:07:57,307 : INFO : Loading a fresh vocabulary
2017-05-11 14:07:57,398 : INFO : min_count=5 retains 13634 unique words (31% of original 43446, drops 29812)
2017-05-11 14:07:57,398 : INFO : min_count=5 leaves 1421890 word corpus (96% of original 1472087, drops 50197)
2017-05-11 14:07:57,438 : INFO : deleting the raw counts dictionary of 43446 items
2017-05-11 14:07:57,441 : INFO : sample=0.001 downsamples 46 most-common words
2017-05-11 14:07:57,442 : INFO : downsampling leaves estimated 1039507 word corpus (73.1% of prior 1421890)
2017-05-11 14:07:57,444 : INFO : estimated

CPU times: user 51.6 s, sys: 242 ms, total: 51.9 s
Wall time: 51.5 s


In [63]:
seed = 1422

doc2vec_model.seed = seed
doc2vec_model.random = random.RandomState(seed)


test_targets, test_regressors = zip(
    *[(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in testsent])

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [65]:
%%time
model_trainer.fit(train_regressors, train_targets)
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.0s finished


CPU times: user 3.59 s, sys: 84.4 ms, total: 3.68 s
Wall time: 21.3 s


In [66]:
yp = model.predict(test_regressors)
yt = test_targets

In [67]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,95,47
1,89,81


In [68]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.52      0.67      0.58       142
          1       0.63      0.48      0.54       170

avg / total       0.58      0.56      0.56       312



In [69]:
accuracy_score(yt, yp)

0.5641025641025641